# RAPL Energy Consumption

**Note:** this notebook is meant to be executed on the supplied Docker image.

In [1]:
import os
from pathlib import Path
import time

This notebook provides excercises related to basic energy consumption monitoring using RAPL. We will read the idle energy consumption of the machine using two interfaces: the RAPL sysfs interface and perf_event interface,

## RAPL sysfs Interface

First of all, it's important to find the zone of your CPU in sysfs. Use the knowledge from the previous modules to find the path of thiz zone and complete the code below.

In [5]:
cpu_zone = '/sys/devices/virtual/powercap/intel-rapl/intel-rapl:0/'

Let's verify whether this zone seems like the zone of your CPU.

In [3]:
if os.path.isdir(cpu_zone):
    print('✓ Zone exists')
else:
    print('✗ Zone does not exist!')

name = Path(f'{cpu_zone}/name').read_text().strip()

if name.startswith('package-'):
    print('✓ Its name starts with \'package-\'')
else:
    print('✗ Its zone does not start with \'package-\'!')

if os.path.isfile(f'{cpu_zone}/energy_uj'):
    print('✓ File \'energy_uj\' exists')
else:
    print('✗ File \'energy_uj\' does not exist!')

✓ Zone exists
✓ Its name starts with 'package-'
✓ File 'energy_uj' exists


The energy counter which reports energy consumed by the zone in micro joules is available as a file named `energy_uj`. This file returns the energy consumed from ar arbitrary, but fixed point in time. By calculating the difference between these values at different points in time, we can obtain the energy consumed during the measurrement period.

Let's start with rading the value of `energy_uj`. Complete the following code.

In [10]:
def energy_uj():
    fp = open(cpu_zone+'energy_uj', 'r')
    return int(fp.read())

print(f'Current energy counter value: {energy_uj()}')

Current energy counter value: 53460156232


Next, we can use the energy counter value read at two different times to calculate energy consumption during the measurement period. Complete the following code. Remembebr that the counter may be reset to zero!

In [17]:
def energy_consumption(energy_uj_start, energy_uj_end):
    fp = open(cpu_zone+'max_energy_range_uj', 'r')
    max_energy_range_uj = int(fp.read())

    if energy_uj_end < energy_uj_start:
        return max_energy_range_uj-energy_uj_start+energy_uj_end
    return energy_uj_end-energy_uj_start

Let's verify whether the function defined above works. Check the output of the script below.

In [18]:
energy = energy_consumption(10000, 10000)
if energy == 0:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

energy = energy_consumption(0, 10000)
if energy == 10000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

counter_max = int(Path(f'{cpu_zone}/max_energy_range_uj').read_text().strip())
energy = energy_consumption(counter_max - 1000, 1000)
if energy == 2000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

✓ Pass
✓ Pass
✓ Pass


Let's try measuring idle CPU energy consumption using the functions `energy_uj()` and `energy_consumption()`. Complete the following code to get the energy consumed during the 5 second measurement period in joules.

In [21]:
measure_time = 5  # seconds

start_energy_uj = energy_uj()
time.sleep(measure_time)
end_energy_uj = energy_uj()

energy_consumed_uj = energy_consumption(start_energy_uj, end_energy_uj)
energy_consumed_j = energy_consumed_uj / 10**6 / measure_time

print(f'Energy consumed: {energy_consumed_j} J')

Energy consumed: 9.3820804 J


The energy consumed should be roughly between 10 and 1000 joules depending on the power of your equipment.